In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import sncosmo
from astropy.table import Table
import matplotlib.pyplot as plt
import warnings

project_root = Path.cwd().parent
print(f"Project root: {project_root}")

Project root: /Users/david/Code/msc


In [2]:
# User input: run folder name
folder_name = input("Enter the run folder name: ").strip()
run_folder = project_root / "runs" / folder_name

lightcurve_files = sorted(run_folder.glob("*_lightcurve.csv"))
print(f"Found {len(lightcurve_files)} lightcurve CSV(s) in {run_folder}")
for f in lightcurve_files:
    print(f"{f.name}")

Enter the run folder name: full_test_1
Found 425 lightcurve CSV(s) in /Users/david/Code/msc/runs/full_test_1
ZTF17aabtvsy_lightcurve.csv
ZTF17aabvong_lightcurve.csv
ZTF17aacldgo_lightcurve.csv
ZTF17aadlxmv_lightcurve.csv
ZTF18aaaonon_lightcurve.csv
ZTF18aaaooqj_lightcurve.csv
ZTF18aaaqexr_lightcurve.csv
ZTF18aadlaxo_lightcurve.csv
ZTF18aadxnul_lightcurve.csv
ZTF18aadzfso_lightcurve.csv
ZTF18aaeqjmc_lightcurve.csv
ZTF18aaermez_lightcurve.csv
ZTF18aafdigb_lightcurve.csv
ZTF18aagkwgz_lightcurve.csv
ZTF18aagtwyh_lightcurve.csv
ZTF18aahfbqp_lightcurve.csv
ZTF18aahtjsc_lightcurve.csv
ZTF18aahvndq_lightcurve.csv
ZTF18aailmnv_lightcurve.csv
ZTF18aaisqmw_lightcurve.csv
ZTF18aaiwzie_lightcurve.csv
ZTF18aaiykoz_lightcurve.csv
ZTF18aaizerg_lightcurve.csv
ZTF18aajivpr_lightcurve.csv
ZTF18aajkcdn_lightcurve.csv
ZTF18aajkrxi_lightcurve.csv
ZTF18aajpjdi_lightcurve.csv
ZTF18aajvqye_lightcurve.csv
ZTF18aakitiq_lightcurve.csv
ZTF18aakiwbs_lightcurve.csv
ZTF18aaklpdo_lightcurve.csv
ZTF18aamvfeb_lightcurve

In [3]:
filter_map = {"g": "ztfg", "r": "ztfr"}
# Redshift per object from ztf_cleansed.csv; fit t0, x0, x1, c only
fit_params = ["t0", "x0", "x1", "c"]
bounds = {"x1": (-3, 3), "c": (-0.3, 0.3)}

all_results = []

In [ ]:
# Load ztf_cleansed.csv
ztf_cleansed_path = project_root / "ztf_cleansed.csv"
ztf_meta = pd.read_csv(ztf_cleansed_path)
ztf_meta["ZTFID"] = ztf_meta["ZTFID"].astype(str).str.strip()

total_files = len(lightcurve_files)
completed = 0

for idx, lc_path in enumerate(lightcurve_files, 1):
    obj_id = lc_path.stem.replace("_lightcurve", "")
    print(f"[{idx}/{total_files}] Processing {obj_id}....")

    # Load cleaned lightcurve from downloadLasair (MJD, filter, forced_ujy, forced_ujy_error)
    df = pd.read_csv(lc_path)
    df["MJD"] = pd.to_numeric(df["MJD"], errors="coerce")
    df = df.dropna(subset=["MJD", "filter", "forced_ujy", "forced_ujy_error"])
    df["filter"] = df["filter"].astype(str).str.strip().str.lower()
    df = df[df["forced_ujy"].gt(0) & df["forced_ujy_error"].gt(0)].copy()

    if len(df) < 5:
        print(f"  Skip {obj_id}: too few points ({len(df)})")
        continue

    bands = [filter_map.get(f, f"ztf{f}") for f in df["filter"].values]
    data = Table({
        "time": df["MJD"].values,
        "band": bands,
        "flux": df["forced_ujy"].values,
        "fluxerr": df["forced_ujy_error"].values,
        "zp": np.full(len(df), 23.9),
        "zpsys": np.array(["ab"] * len(df)),
    })

    # Look up obj_id in the ZTFID column
    meta_row = ztf_meta.loc[ztf_meta["ZTFID"] == obj_id].iloc[0]
    redshift = float(meta_row["redshift"])
    A_V = float(pd.to_numeric(meta_row.get("A_V", 0), errors="coerce") or 0)
    ebv = A_V / 3.1  # E(B-V) from A_V with R_V = 3.1 (standard value for MW)

    # SALT2 + Milky Way dust (observer frame) using A_V from ztf_cleansed
    # effect_names=['mw'] gives parameter 'mwebv' (prefix 'mw' + dust param 'ebv')
    dust = sncosmo.F99Dust()
    model = sncosmo.Model(source="salt2", effects=[dust], effect_names=["mw"], effect_frames=["obs"])
    model.set(z=redshift, mwebv=ebv)

    try:
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", RuntimeWarning)
            result, fitted_model = sncosmo.fit_lc(data, model, fit_params, bounds=bounds)
    except Exception as e:
        print(f"  Fit failed: {e}")
        continue

    row = {"object_id": obj_id, "chisq": result.chisq, "ndof": result.ndof, "z": redshift}
    for name, val in zip(result.param_names, result.parameters):
        row[name] = val
    if result.errors is not None:
        for name in result.param_names:
            if name in result.errors:
                row[f"{name}_err"] = result.errors[name]
    all_results.append(row)

    fig = sncosmo.plot_lc(data, model=fitted_model, errors=result.errors)
    plt.gcf().suptitle(obj_id)
    out_plot = run_folder / f"{lc_path.stem}_salt2.png"
    plt.savefig(out_plot, dpi=150, bbox_inches="tight")
    plt.close()
    completed += 1
    print(f"  Saved {out_plot.name}  ({completed}/{total_files} completed)")

[1/425] Processing ZTF17aabtvsy....
  Saved ZTF17aabtvsy_lightcurve_salt2.png  (1/425 completed)
[2/425] Processing ZTF17aabvong....
  Saved ZTF17aabvong_lightcurve_salt2.png  (2/425 completed)
[3/425] Processing ZTF17aacldgo....
  Saved ZTF17aacldgo_lightcurve_salt2.png  (3/425 completed)
[4/425] Processing ZTF17aadlxmv....
  Saved ZTF17aadlxmv_lightcurve_salt2.png  (4/425 completed)
[5/425] Processing ZTF18aaaonon....
  Saved ZTF18aaaonon_lightcurve_salt2.png  (5/425 completed)
[6/425] Processing ZTF18aaaooqj....
  Saved ZTF18aaaooqj_lightcurve_salt2.png  (6/425 completed)
[7/425] Processing ZTF18aaaqexr....
  Saved ZTF18aaaqexr_lightcurve_salt2.png  (7/425 completed)
[8/425] Processing ZTF18aadlaxo....
  Saved ZTF18aadlaxo_lightcurve_salt2.png  (8/425 completed)
[9/425] Processing ZTF18aadxnul....
  Skip ZTF18aadxnul: too few points (3)
[10/425] Processing ZTF18aadzfso....
  Saved ZTF18aadzfso_lightcurve_salt2.png  (9/425 completed)
[11/425] Processing ZTF18aaeqjmc....
  Saved ZTF18

  Saved ZTF18aavsilo_lightcurve_salt2.png  (83/425 completed)
[85/425] Processing ZTF18aavvnzu....
  Saved ZTF18aavvnzu_lightcurve_salt2.png  (84/425 completed)
[86/425] Processing ZTF18aawfqax....
  Saved ZTF18aawfqax_lightcurve_salt2.png  (85/425 completed)
[87/425] Processing ZTF18aawglif....
  Saved ZTF18aawglif_lightcurve_salt2.png  (86/425 completed)
[88/425] Processing ZTF18aawiilg....
  Saved ZTF18aawiilg_lightcurve_salt2.png  (87/425 completed)
[89/425] Processing ZTF18aawitoj....
  Saved ZTF18aawitoj_lightcurve_salt2.png  (88/425 completed)
[90/425] Processing ZTF18aawitry....
  Saved ZTF18aawitry_lightcurve_salt2.png  (89/425 completed)
[91/425] Processing ZTF18aawjywv....
  Saved ZTF18aawjywv_lightcurve_salt2.png  (90/425 completed)
[92/425] Processing ZTF18aawlfnj....
  Saved ZTF18aawlfnj_lightcurve_salt2.png  (91/425 completed)
[93/425] Processing ZTF18aawlxls....
  Saved ZTF18aawlxls_lightcurve_salt2.png  (92/425 completed)
[94/425] Processing ZTF18aawmeiw....
  Saved ZT

  Saved ZTF18abfeyey_lightcurve_salt2.png  (164/425 completed)
[167/425] Processing ZTF18abffdjl....
  Saved ZTF18abffdjl_lightcurve_salt2.png  (165/425 completed)
[168/425] Processing ZTF18abffdpa....
  Saved ZTF18abffdpa_lightcurve_salt2.png  (166/425 completed)
[169/425] Processing ZTF18abfgygp....
  Saved ZTF18abfgygp_lightcurve_salt2.png  (167/425 completed)
[170/425] Processing ZTF18abfhaji....
  Saved ZTF18abfhaji_lightcurve_salt2.png  (168/425 completed)
[171/425] Processing ZTF18abfhryc....
  Saved ZTF18abfhryc_lightcurve_salt2.png  (169/425 completed)
[172/425] Processing ZTF18abfwuwn....
  Saved ZTF18abfwuwn_lightcurve_salt2.png  (170/425 completed)
[173/425] Processing ZTF18abgkjff....
  Saved ZTF18abgkjff_lightcurve_salt2.png  (171/425 completed)
[174/425] Processing ZTF18abgqkpc....
  Saved ZTF18abgqkpc_lightcurve_salt2.png  (172/425 completed)
[175/425] Processing ZTF18abgxkzx....
  Saved ZTF18abgxkzx_lightcurve_salt2.png  (173/425 completed)
[176/425] Processing ZTF18ab

  Saved ZTF18acaezsx_lightcurve_salt2.png  (246/425 completed)
[250/425] Processing ZTF18acbvcmw....
  Saved ZTF18acbvcmw_lightcurve_salt2.png  (247/425 completed)
[251/425] Processing ZTF18acbvgqw....
  Saved ZTF18acbvgqw_lightcurve_salt2.png  (248/425 completed)
[252/425] Processing ZTF18acbxmvy....
  Saved ZTF18acbxmvy_lightcurve_salt2.png  (249/425 completed)
[253/425] Processing ZTF18acbxsap....
  Saved ZTF18acbxsap_lightcurve_salt2.png  (250/425 completed)
[254/425] Processing ZTF18acbxsge....
  Saved ZTF18acbxsge_lightcurve_salt2.png  (251/425 completed)
[255/425] Processing ZTF18acbyyff....
  Saved ZTF18acbyyff_lightcurve_salt2.png  (252/425 completed)
[256/425] Processing ZTF18accjdgs....
  Saved ZTF18accjdgs_lightcurve_salt2.png  (253/425 completed)
[257/425] Processing ZTF18acclexy....
  Saved ZTF18acclexy_lightcurve_salt2.png  (254/425 completed)
[258/425] Processing ZTF18accndxn....
  Saved ZTF18accndxn_lightcurve_salt2.png  (255/425 completed)
[259/425] Processing ZTF18ac

In [23]:
# Output CSV of sncosmo parameters
if all_results:
    params_df = pd.DataFrame(all_results)
    out_csv = run_folder / "sncosmo_parameters.csv"
    params_df.to_csv(out_csv, index=False)
    print(f"Saved sncosmo parameters to {out_csv}")
    display(params_df)
else:
    print("No successful fits to write.")

Saved sncosmo parameters to /Users/david/Code/msc/runs/test4/sncosmo_parameters.csv


,object_id,chisq,ndof,z,t0,x0,x1,c,mwebv,t0_err,x0_err,x1_err,c_err
0,ZTF19aailqir,1.707646,7,0.05434,58529.181776,0.000451,0.139796,0.300000,0.024839,0.539645,0.000021,0.731697,0.413524
1,ZTF19aailqli,86.067663,19,0.05800,58535.586380,0.001421,0.197938,-0.039584,0.013871,0.240893,0.000094,0.368545,0.047733
2,ZTF19aailteb,16.736875,8,0.07800,58524.495481,0.000763,2.999994,-0.036656,0.014516,0.835964,0.000040,0.974345,0.053904
3,ZTF19aailuvj,4.448203,14,0.07100,58536.547675,0.000903,0.470430,-0.094157,0.021290,0.313551,0.000059,0.422184,0.047706
4,ZTF19aajwdqt,16.834560,13,0.08800,58538.834603,0.000837,3.000000,0.029961,0.046129,0.333009,0.000039,0.171656,0.046427
5,ZTF19aajwhbc,0.253676,1,0.10000,58536.914900,0.000563,-0.571901,-0.139275,0.013871,2.106258,0.000348,4.021764,0.339512
6,ZTF19aajxhwa,14.489825,18,0.03300,58539.506247,0.003617,0.403831,-0.061157,0.206129,0.229294,0.000106,0.298906,0.028861
7,ZTF19aajxwnz,2948.340692,73,0.01500,58544.708295,0.020554,0.849131,0.042802,0.154194,0.066075,0.000327,0.084007,0.011186
8,ZTF19aakiwxx,6.581250,30,0.04500,58537.073989,0.002080,-1.004063,-0.063543,0.016452,0.172475,0.000072,0.175499,0.028095
9,ZTF19aakiyly,57.217998,17,0.07000,58539.562164,0.001059,0.666689,-0.072604,0.028065,0.307995,0.000049,0.304747,0.038799
